# Testing src functions

In [1]:
from src import models, datasets, trainer
import torch

In [20]:
testrnn = models.RNN(1, 5, 1, 0.1, 50, 'relu', bias=False)

In [13]:
cur_hidden = torch.zeros(5)

In [15]:
hidden = testrnn.forward(torch.ones(1), cur_hidden)

In [18]:
testrnn.ho(hidden)

tensor([-0.0438], grad_fn=<ViewBackward0>)

In [20]:
import numpy as np

In [23]:
t = np.arange(0, 20.1, 0.1)
stim_bins = np.where((t>=1)&(t<=(3)))

In [26]:
t[stim_bins]

array([1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2,
       2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. ])

In [19]:
testdata = datasets.decisionMakingInstant(0, 1, 3, 3, 3)

In [29]:
testrnn.hh.weight

Parameter containing:
tensor([[-2.7093e-01, -3.3656e-04,  1.5870e-01, -8.2846e-02, -2.0841e-01],
        [-4.4533e-01,  3.6139e-01, -4.1310e-01, -3.9851e-01,  1.9130e-01],
        [ 7.7235e-02,  4.1670e-01, -2.1953e-01,  2.9130e-01,  3.5128e-01],
        [ 1.8750e-01, -1.9956e-01, -2.4311e-01,  1.9028e-01, -1.5286e-01],
        [ 2.8113e-01,  3.0368e-01,  3.9393e-01,  3.4493e-04, -2.7769e-02]],
       requires_grad=True)

In [31]:
rnn = torch.load('0_best_RNN.pth')

/var/folders/48/lxr9wxxd4yq06qqj9jrz69780000gt/T/ipykernel_81153/1829952402.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rnn = torch.load('0_best_RNN.pth')


In [ ]:
rnn[]

OrderedDict([('ih.weight',
              tensor([[ 0.7530],
                      [ 0.8486],
                      [-0.8472],
                      [-0.6936],
                      [ 0.8855],
                      [-0.0167],
                      [-0.9347],
                      [-1.4740],
                      [-0.5561],
                      [ 1.4302],
                      [-0.6652],
                      [-0.6771],
                      [-0.9832],
                      [-0.7691],
                      [ 0.9426],
                      [-0.9688],
                      [ 0.0335],
                      [ 0.7604],
                      [ 0.8888],
                      [-0.2192],
                      [-1.2275],
                      [ 0.6366],
                      [-0.6573],
                      [-0.3079],
                      [-0.1635],
                      [ 0.1093],
                      [-0.5730],
                      [ 0.1510],
                      [ 0.4325],
                